## constants.py

In [1]:
from torchvision import transforms

# Data Augmentation defaults
TRAIN_TRANSFORMS = transforms.Compose([
            transforms.Resize(32),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ])

TEST_TRANSFORMS = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor()
    ])

## fine_tunify.py

In [2]:
!pip install robustness

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 1.6 MB/s 
     |████████████████████████████████| 125 kB 3.3 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 184 kB 12.7 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=22376870f57f081ffc435b0ce296fbd20c8b8c2f38fd0acec47011b57d774f57
  Stored in directory: /root/.cache/pip/wheels/ba/03/bb/7a97840eb54479b328672e15a536e49dc60da200fb21564d53
Successfully built GPUtil


In [2]:
from torch import nn
from robustness.tools.custom_modules import SequentialWithArgs

def ft(model_name, model_ft, num_classes, additional_hidden=0):
    if model_name in ["resnet", "resnet18", "resnet50", "wide_resnet50_2", "wide_resnet50_4", "wide_resnet101_2", "resnext50_32x4d", 'shufflenet']:
        num_ftrs = model_ft.fc.in_features
        # The two cases are split just to allow loading
        # models trained prior to adding the additional_hidden argument
        # without errors
        if additional_hidden == 0:
            model_ft.fc = nn.Linear(num_ftrs, num_classes)
        else:
            model_ft.fc = SequentialWithArgs(
                *list(sum([[nn.Linear(num_ftrs, num_ftrs), nn.ReLU()] for i in range(additional_hidden)], [])),
                nn.Linear(num_ftrs, num_classes)
            )
        input_size = 224
    elif 'regnet' in model_name:
        num_ftrs = model_ft.head.fc.in_features
        model_ft.head.fc = nn.Linear(num_ftrs, num_classes)
    elif 'deit' in model_name:
        num_ftrs = model_ft.head.in_features
        model_ft.head = nn.Linear(num_ftrs, num_classes)
        input_size = 224 
    elif model_name == "alexnet":
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
    elif "vgg" in model_name:
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
    elif model_name == "squeezenet":
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224
    elif model_name == "densenet":
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name in ["mnasnet", "mobilenet"]:
        num_ftrs = model_ft.classifier[1].in_features
        model_ft.classifier[1] = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    else:
        raise ValueError("Invalid model type, exiting...")

    return model_ft

## solution.py

In [3]:
import torch as ch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
import os
import itertools
import math

def ablate(x, pos, k, total_pos, dim): 
    # x : input
    # pos : starting position
    # k : size of ablation
    # total_pos : maximum position
    # dim : height or width (2 or 3)
    inp = ch.zeros_like(x)
    mask = x.new_zeros(x.size(0), 1, x.size(2), x.size(3))
    if pos + k > total_pos: 
        idx1 = [slice(None,None,None) if _ != dim else slice(pos,total_pos,None) for _ in range(4)]
        idx2 = [slice(None,None,None) if _ != dim else slice(0,pos+k-total_pos,None) for _ in range(4)]
        inp[idx1] = x[idx1]
        inp[idx2] = x[idx2]
        mask[idx1] = 1
        mask[idx2] = 1
    else: 
        idx = [slice(None,None,None) if _ != dim else slice(pos,pos+k,None) for _ in range(4)]
        inp[idx] = x[idx]
        mask[idx] = 1
    return ch.cat([inp,mask],dim=1)

def ablate2(x,block_pos,block_k,shape): 
    inp = ch.zeros_like(x)
    mask = x.new_zeros(x.size(0), 1, x.size(2), x.size(3))

    slices = []
    for pos,k,total_pos in zip(block_pos,block_k,shape): 
        if pos + k > total_pos: 
            slices.append([slice(0,pos+k-total_pos,None), slice(pos,total_pos,None)])
        else: 
            slices.append([slice(pos,pos+k,None)])

    for si,sj in itertools.product(*slices): 
        idx = [slice(None,None,None),slice(None,None,None),si,sj]
        inp[idx] = x[idx]
        mask[idx] = 1

    return ch.cat([inp,mask],dim=1)
    

class DerandomizedSmoother(nn.Module): 
    def __init__(self, column_model=None, row_model=None, block_size=(4,4), stride=(1,1), preprocess=None): 
        super(DerandomizedSmoother, self).__init__()
        self.column_model = column_model
        self.row_model = row_model
        self.block_size = block_size
        self.stride = stride
        self.preprocess = preprocess
        
    def forward(self, x, nclasses=10, threshold=None, return_mode=None): 
        # return_mode == 'differentiable', 'ablations', 'predictions'
        nex, nch, h, w = x.size()
        
        predictions = x.new_zeros(nex, nclasses)
        softmaxes = 0
        ablations = []
        for model, total_pos, k, s, dim in zip((self.row_model, self.column_model), 
                                               (h,w), 
                                               self.block_size, 
                                               self.stride, 
                                               (2,3)): 
            if model is not None: 
                for pos in range(0,total_pos,s): 
                    inp = ablate(x, pos, k, total_pos, dim)
                    if self.preprocess is not None: 
                        inp = self.preprocess(inp)
                    out = model(inp)
                    if isinstance(out, tuple): 
                        out = out[0]
                    out = F.softmax(out,dim=1)

                    if return_mode == 'differentiable': 
                        softmaxes += out
                    if return_mode == 'ablations' or return_mode == 'all': 
                        ablations.append(out.max(1)[1].unsqueeze(1))

                    if threshold is not None: 
                        predictions += (out >= threshold).int()
                    else: 
                        predictions += (out.max(1)[0].unsqueeze(1) == out).int()
        
        if return_mode == 'differentiable': 
            return softmaxes/len(range(0,total_pos,s))
        if return_mode == 'predictions': 
            return predictions.argmax(1), predictions
        if return_mode == 'ablations': 
            return predictions.argmax(1), ch.cat(ablations,dim=1)
        if return_mode == 'all': 
            return predictions.argmax(1), predictions, ch.cat(ablations,dim=1)

        return predictions.argmax(1)

class BlockDerandomizedSmoother(nn.Module): 
    def __init__(self, block_model=None, block_size=(4,4), stride=(1,1), preprocess=None): 
        super(BlockDerandomizedSmoother, self).__init__()
        self.model = block_model
        self.block_size = block_size
        self.stride = stride
        self.preprocess = preprocess
        
    def forward(self, x, nclasses=10, threshold=None, return_mode=None): 
        # return_mode == 'differentiable', 'ablations', 'predictions'
        nex, nch, h, w = x.size()
        
        predictions = x.new_zeros(nex, nclasses)
        softmaxes = 0
        ablations = []

        for i_pos in tqdm(range(0,h,self.stride[0])): 
            for j_pos in range(0,w,self.stride[1]): 
                inp = ablate2(x, (i_pos,j_pos), self.block_size, (h,w))
                if self.preprocess is not None: 
                    inp = self.preprocess(inp)
                out = self.model(inp)
                if isinstance(out, tuple): 
                    out = out[0]
                out = F.softmax(out,dim=1)

                if return_mode == 'differentiable': 
                    softmaxes += out
                if return_mode == 'ablations' or return_mode == 'all': 
                    ablations.append(out.max(1)[1].unsqueeze(1))

                if threshold is not None: 
                    predictions += (out >= threshold).int()
                else: 
                    predictions += (out.max(1)[0].unsqueeze(1) == out).int()
        
        if return_mode == 'differentiable': 
            return softmaxes/len(range(0,total_pos,s))
        if return_mode == 'predictions': 
            return predictions.argmax(1), predictions
        if return_mode == 'ablations': 
            return predictions.argmax(1), ch.cat(ablations,dim=1)
        if return_mode == 'all': 
            return predictions.argmax(1), predictions, ch.cat(ablations,dim=1)


        return predictions.argmax(1)

def certify(args, model, validation_loader, store=None): 
    # print("Certification is replacing transform with ToTensor")
    m = args.certify_patch_size
    s = args.certify_ablation_size
    stride = args.certify_stride

    if args.dataset == 'cifar10': 
        nclasses = 10
    elif args.dataset == 'imagenet': 
        nclasses = 1000
    else: 
        raise ValueError("Unknown number of classes")

    os.makedirs(args.certify_out_dir, exist_ok=True)
    os.makedirs(os.path.join(args.certify_out_dir, args.exp_name), exist_ok=True)
    summary_path = os.path.join(args.certify_out_dir,args.exp_name,f"m{m}_s{s}_summary.pth")
    if os.path.exists(summary_path): 
        d = ch.load(summary_path)
        print("summary:")
        print(f"acc: {d['acc']:.4f}, abl {d['ablation_acc']:.4f},  cert {d['cert_acc']:.4f}, delta: {d['delta']:.4f}, s: {s}, m: {m}")
        return d['delta']

    model.eval() 
    model = nn.DataParallel(model)
    with ch.no_grad(): 
        col_model = model if args.certify_mode in ['both', 'col'] else None
        row_model = model if args.certify_mode in ['both', 'row'] else None

        # number of ablations in one axis
        na = math.ceil((m + s - 1)/stride)
        if args.certify_mode == 'block': 
            smoothed_model = BlockDerandomizedSmoother(
                block_model=model, 
                block_size=(s,s), 
                stride=(stride,stride)
            )
            gap = 2*(na**2) + 1
        else: 
            smoothed_model = DerandomizedSmoother(
                column_model=col_model, 
                row_model=row_model, 
                block_size=(s,s), 
                stride=(stride,stride)
            )
            # add one to not handle ties
            # 2*(m + s - 1) for one dimension of ablations, and 
            # double again for two axes
            factor = 4 if args.certify_mode == 'both' else 2
            gap = na*factor + 1 

        total = 0
        n = 0
        smooth_total = 0
        certified_total = 0
        ablation_total = 0
        delta = 0
        

        pbar = tqdm(validation_loader)
        for i,(X,y) in enumerate(pbar): 
            if args.batch_id != None and args.batch_id < i: 
                break
            if args.batch_id != None and args.batch_id != i: 
                continue
            file_path = os.path.join(args.certify_out_dir,args.exp_name,f"m{m}_s{s}_b{i}.pth")
            if os.path.exists(file_path): 
                d = ch.load(file_path)

                smooth_total += d['smooth_delta']
                ablation_total += d['ablation_delta']
                certified_total += d['certified_delta']
                total += d['total_delta']
                delta += d['delta_delta']
                n += X.size(0)

                pbar.set_description(f"Acc: {total/n:.4f} Abl acc: {ablation_total/n:.4f} Smo acc: {smooth_total/n:.4f} Cer acc: {certified_total/n:.4f} Delta: {delta/n:.0f}")
                continue
            X,y = X.cuda(),y.cuda()
            acc = (model(X)[0].max(1)[1] == y).float().mean()

            y_smoothed, y_counts, y_ablations = smoothed_model(X, return_mode="all", nclasses=nclasses)
            y_1st_vals, y_1st_idx = y_counts.kthvalue(nclasses,dim=1)
            y_2nd_vals, y_2nd_idx = y_counts.kthvalue(nclasses-1,dim=1)

            y_tar_vals = ch.gather(y_counts,1,y.unsqueeze(1)).squeeze()
            not_y = (y_1st_idx != y)
            y_nex_idx = y_1st_idx*(not_y.int()) + y_2nd_idx*(~not_y)
            y_nex_vals = ch.gather(y_counts,1,y_nex_idx.unsqueeze(1)).squeeze()
            
            y_certified = (y == y_1st_idx)*(y_1st_vals >= y_2nd_vals + gap)

            smooth_delta = (y_smoothed == y).sum().item()
            smooth_total += smooth_delta

            ablation_delta = y_tar_vals.sum().item()
            ablation_total += ablation_delta

            certified_delta = y_certified.sum().item()
            certified_total += certified_delta

            total_delta = acc.item()*X.size(0)
            total += total_delta

            delta_delta = (y_tar_vals - y_nex_vals).sum().item()
            delta += delta_delta
            n += X.size(0)

            ch.save({
                "total_delta" : total_delta, 
                "certified_delta" : certified_delta, 
                "smooth_delta": smooth_delta, 
                "ablation_delta": ablation_delta, 
                "delta_delta": delta_delta, 
                "s": s, 
                "m": m, 
                "mode": args.certify_mode, 
                "ablations": y_ablations.detach().cpu(), 
                "y": y.cpu()
            }, file_path)

            pbar.set_description(f"Acc: {total/n:.4f} Abl acc: {ablation_total/n:.4f} Smo acc: {smooth_total/n:.4f} Cer acc: {certified_total/n:.4f} Delta: {delta/n:.0f}")

        if args.batch_id == None: 
            ch.save({
                "acc" : total/n, 
                "cert_acc" : certified_total/n, 
                "smooth_acc": smooth_total/n, 
                "ablation_acc": ablation_total/n, 
                "delta": delta/n, 
                "s": s, 
                "m": m, 
                "mode": args.certify_mode
            }, summary_path)

            print(f"acc: {total/n:.4f}, ablation {ablation_total/n:.4f}, smoothed {smooth_total/n:.4f}, certified {certified_total/n:.4f}, delta: {delta/n:.4f}, s: {s}, m: {m}")
        return delta/n

## vision_transformer.py

In [4]:
""" Vision Transformer (ViT) in PyTorch
A PyTorch implement of Vision Transformers as described in
'An Image Is Worth 16 x 16 Words: Transformers for Image Recognition at Scale' - https://arxiv.org/abs/2010.11929
The official jax code is released and available at https://github.com/google-research/vision_transformer
Status/TODO:
* Models updated to be compatible with official impl. Args added to support backward compat for old PyTorch weights.
* Weights ported from official jax impl for 384x384 base and small models, 16x16 and 32x32 patches.
* Trained (supervised on ImageNet-1k) my custom 'small' patch model to 77.9, 'base' to 79.4 top-1 with this code.
* Hopefully find time and GPUs for SSL or unsupervised pretraining on OpenImages w/ ImageNet fine-tune in future.
Acknowledgments:
* The paper authors for releasing code and weights, thanks!
* I fixed my class token impl based on Phil Wang's https://github.com/lucidrains/vit-pytorch ... check it out
for some einops/einsum fun
* Simple transformer style inspired by Andrej Karpathy's https://github.com/karpathy/minGPT
* Bert reference code checks against Huggingface Transformers and Tensorflow Bert
Hacked together by / Copyright 2020 Ross Wightman
"""
import torch
import torch.nn as nn
from functools import partial

# from .resnet import resnet26d, resnet50d
from enum import Enum

IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)


class Debug_Flags(Enum):
    L2_HEAD_CTX_NORMS = 1 # return the l2 norms of the context heads
    ATTN_MASKS = 2 # return the attention masks
    HEAD_CTX = 3 # return the head contexts
    LAYER_CTX = 4 # return the aggregated head contexts
    HEAD_OUTPUT = 5 # return the attention output from each head
    LAYER_OUTPUT = 6 # return the output from aggregated heads
    RESIDUAL_CTX_VEC = 7 # residual before adding layer output
    RESIDUAL_CTX_ADD = 8 # ctx right before adding to residual
    RESIDUAL_CTX_FINAL = 9 # RESIDUAL_VEC + RESIDUAL_ADD
    FINAL_LATENT_VECTOR = 10 # final latent vector for the model
    PATCH_EMBED = 11
    RESIDUAL_LAYER_FINAL = 12 # RESIDUAL_VEC + RESIDUAL_ADD but for the whole raster
    NO_POS = 13


def _cfg(url='', **kwargs):
    return {
        'url': url,
        'num_classes': 1000, 'input_size': (3, 224, 224), 'pool_size': None,
        'crop_pct': .9, 'interpolation': 'bicubic',
        'mean': IMAGENET_DEFAULT_MEAN, 'std': IMAGENET_DEFAULT_STD,
        'first_conv': 'patch_embed.proj', 'classifier': 'head',
        **kwargs
    }


default_cfgs = {
    # patch models
    'vit_small_patch16_224': _cfg(
        url='https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/vit_small_p16_224-15ec54c9.pth',
    ),
    'vit_base_patch16_224': _cfg(
        url='https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
    'vit_base_patch16_384': _cfg(
        url='https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_384-83fb41ba.pth',
        input_size=(3, 384, 384), mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), crop_pct=1.0),
    'vit_base_patch32_384': _cfg(
        url='https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p32_384-830016f5.pth',
        input_size=(3, 384, 384), mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), crop_pct=1.0),
    'vit_large_patch16_224': _cfg(
        url='https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_large_p16_224-4ee7a4dc.pth',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    'vit_large_patch16_384': _cfg(
        url='https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_large_p16_384-b3be5167.pth',
        input_size=(3, 384, 384), mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), crop_pct=1.0),
    'vit_large_patch32_384': _cfg(
        url='https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_large_p32_384-9b920ba8.pth',
        input_size=(3, 384, 384), mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), crop_pct=1.0),
    'vit_huge_patch16_224': _cfg(),
    'vit_huge_patch32_384': _cfg(input_size=(3, 384, 384)),
    # hybrid models
    'vit_small_resnet26d_224': _cfg(),
    'vit_small_resnet50d_s3_224': _cfg(),
    'vit_base_resnet26d_224': _cfg(),
    'vit_base_resnet50d_224': _cfg(),
    # deit models
    'deit_tiny_patch16_224': _cfg(
        url='https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth',
        crop_pct=0.875, interpolation=3
    ),
    'deit_small_patch16_224': _cfg(
        url='https://dl.fbaipublicfiles.com/deit/deit_small_patch16_224-cd65a155.pth',
        crop_pct=0.875, interpolation=3
    ),
    'deit_base_patch16_224': _cfg(
        url='https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth',
        crop_pct=0.875, interpolation=3
    ),
    'deit_base_patch16_384': _cfg(
        crop_pct=1.0, interpolation=4,  input_size=(3, 384, 384),
    ),
}


class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x, head_hide_list=None, debug_flags=None):
        debug = {}
        if debug_flags is None:
            debug_flags = {}

        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = (attn - torch.max(attn, dim=-1, keepdim=True)[0]).softmax(dim=-1)
        if head_hide_list is not None:
            # we are masking some heads
            if -1 in head_hide_list:
                # mask all in this layer
                attn[:, :, :, 1:] *= 0
                attn[:, :, :, 0] = 1
            else:
                attn[:, head_hide_list, :, 1:] *= 0
                attn[:, head_hide_list, :, 0] = 1

        if Debug_Flags.ATTN_MASKS in debug_flags:
            debug[Debug_Flags.ATTN_MASKS.name] = attn # store attention mask

        attn = self.attn_drop(attn)
        x = (attn @ v).transpose(1, 2) # B, N, Num Heads, head_dim
        if Debug_Flags.L2_HEAD_CTX_NORMS in debug_flags:
            l2_norms = x[:, 0].norm(dim=-1)
            debug[Debug_Flags.L2_HEAD_CTX_NORMS.name] = l2_norms
        if Debug_Flags.HEAD_CTX in debug_flags:
            head_contexts = x[:, 0]
            debug[Debug_Flags.HEAD_CTX.name] = head_contexts
        if Debug_Flags.HEAD_OUTPUT in debug_flags:
            head_outputs = x
            debug[Debug_Flags.HEAD_OUTPUT.name] = head_outputs
        
        x = x.reshape(B, N, C)
        x = self.proj(x)
        if Debug_Flags.LAYER_CTX in debug_flags:
            layer_contexts = x[:, 0]
            debug[Debug_Flags.LAYER_CTX.name] = layer_contexts
        if Debug_Flags.LAYER_OUTPUT in debug_flags:
            layer_outputs = x
            debug[Debug_Flags.LAYER_OUTPUT.name] = layer_outputs
        x = self.proj_drop(x)
        return x, debug


class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x, head_hide_list=None, debug_flags=None):
        if debug_flags is None:
            debug_flags = {}
        out, debug = self.attn(self.norm1(x), head_hide_list=head_hide_list, debug_flags=debug_flags)
        out = self.drop_path(out)
        if Debug_Flags.RESIDUAL_CTX_ADD in debug_flags:
            debug[Debug_Flags.RESIDUAL_CTX_ADD.name] = out[:, 0]
        if Debug_Flags.RESIDUAL_CTX_VEC in debug_flags:
            debug[Debug_Flags.RESIDUAL_CTX_VEC.name] = x[:, 0]
        x = x + out
        if Debug_Flags.RESIDUAL_CTX_FINAL in debug_flags:
            debug[Debug_Flags.RESIDUAL_CTX_FINAL.name] = x[:, 0]
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        if Debug_Flags.RESIDUAL_LAYER_FINAL in debug_flags:
            debug[Debug_Flags.RESIDUAL_LAYER_FINAL.name] = x
        return x, debug


class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        # FIXME look at relaxing size constraints
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x


class HybridEmbed(nn.Module):
    """ CNN Feature Map Embedding
    Extract feature map from CNN, flatten, project to embedding dim.
    """
    def __init__(self, backbone, img_size=224, feature_size=None, in_chans=3, embed_dim=768):
        super().__init__()
        assert isinstance(backbone, nn.Module)
        img_size = to_2tuple(img_size)
        self.img_size = img_size
        self.backbone = backbone
        if feature_size is None:
            with torch.no_grad():
                # FIXME this is hacky, but most reliable way of determining the exact dim of the output feature
                # map for all networks, the feature metadata has reliable channel and stride info, but using
                # stride to calc feature dim requires info about padding of each stage that isn't captured.
                training = backbone.training
                if training:
                    backbone.eval()
                o = self.backbone(torch.zeros(1, in_chans, img_size[0], img_size[1]))[-1]
                feature_size = o.shape[-2:]
                feature_dim = o.shape[1]
                backbone.train(training)
        else:
            feature_size = to_2tuple(feature_size)
            feature_dim = self.backbone.feature_info.channels()[-1]
        self.num_patches = feature_size[0] * feature_size[1]
        self.proj = nn.Linear(feature_dim, embed_dim)

    def forward(self, x):
        x = self.backbone(x)[-1]
        x = x.flatten(2).transpose(1, 2)
        x = self.proj(x)
        return x


class VisionTransformer(nn.Module):
    """ Vision Transformer with support for patch or hybrid CNN input stage
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=1000, embed_dim=768, depth=12,
                 num_heads=12, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop_rate=0., attn_drop_rate=0.,
                 drop_path_rate=0., hybrid_backbone=None, norm_layer=nn.LayerNorm, norm_embed=False):
        super().__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models

        if hybrid_backbone is not None:
            self.patch_embed = HybridEmbed(
                hybrid_backbone, img_size=img_size, in_chans=in_chans, embed_dim=embed_dim)
        else:
            self.patch_embed = PatchEmbed(
                img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)

        # self.norm_embed = norm_layer(embed_dim) if norm_embed else None
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule
        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        # NOTE as per official impl, we could have a pre-logits representation dense layer + tanh here
        #self.repr = nn.Linear(embed_dim, representation_size)
        #self.repr_act = nn.Tanh()

        # Classifier head
        self.head = nn.Linear(embed_dim, num_classes) if num_classes > 0 else nn.Identity()

        trunc_normal_(self.pos_embed, std=.02)
        trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)
        self.mask_processor = MaskProcessor(patch_size=patch_size)
        self.first_pass = True

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'pos_embed', 'cls_token'}

    def get_classifier(self):
        return self.head

    def reset_classifier(self, num_classes, global_pool=''):
        self.num_classes = num_classes
        self.head = nn.Linear(self.embed_dim, num_classes) if num_classes > 0 else nn.Identity()

    def forward_features(self, x, head_mask, debug_flags, patch_mask): 
        if x.shape[1] == 4:
            if self.first_pass:
                print("USING MISSINGNESS")
            self.first_pass = False
            assert patch_mask is None
            x, ones_mask = x[:, :3], x[:, 3]
            patch_mask = self.mask_processor(ones_mask)
            
        B = x.shape[0]
        x = self.patch_embed(x)
        patch_embedding = x
        if head_mask is None:
            head_mask = {}

        # if self.norm_embed:
        #     x = self.norm_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
        x = torch.cat((cls_tokens, x), dim=1)
        if debug_flags is not None and Debug_Flags.NO_POS in debug_flags:
            x = x
        else:
            x = x + self.pos_embed
        x = self.pos_drop(x) # B, N, C
        if patch_mask is not None:
            # patch_mask is B, K
            B, N, C = x.shape
            if len(patch_mask.shape) == 1: # not a separate one per batch
                x = x[:, patch_mask]
            else:
                patch_mask = patch_mask.unsqueeze(-1).expand(-1, -1, C)
                x = torch.gather(x, 1, patch_mask)

        all_debug = []
        for i, blk in enumerate(self.blocks):
            if i in head_mask:
                x, all_debug_layer = blk(x, head_hide_list=head_mask[i], debug_flags=debug_flags)
            else:
                x, all_debug_layer = blk(x, debug_flags=debug_flags)
            all_debug.append(all_debug_layer)

        consolidated_all_debug = {}
        for e in all_debug[0].keys():
            consolidated_all_debug[e] = torch.stack([d[e] for d in all_debug], 1)
        
        if debug_flags is not None and Debug_Flags.PATCH_EMBED in debug_flags:
            consolidated_all_debug[Debug_Flags.PATCH_EMBED.name] = patch_embedding

        x = self.norm(x)
        return x[:, 0], consolidated_all_debug

    def forward(self, x, head_mask=None, debug_flags=None, patch_mask=None):
        # dict of layer_index -> list of head indices to turn off. If list just contains -1, turn all heads off in that layer
        x, debug = self.forward_features(x, head_mask=head_mask, debug_flags=debug_flags, patch_mask=patch_mask)
        if debug_flags is not None and Debug_Flags.FINAL_LATENT_VECTOR in debug_flags:
            debug[Debug_Flags.FINAL_LATENT_VECTOR.name] = x
        x = self.head(x)
        if debug_flags is None:
            return x
        else:
            return x, debug


def _conv_filter(state_dict, patch_size=16):
    """ convert patch embedding weight from manual patchify + linear proj to conv"""
    out_dict = {}
    for k, v in state_dict.items():
        if 'patch_embed.proj.weight' in k:
            v = v.reshape((v.shape[0], 3, patch_size, patch_size))
        out_dict[k] = v
    return out_dict


def vit_small_patch16_224(pretrained=False, **kwargs):
    if pretrained:
        # NOTE my scale was wrong for original weights, leaving this here until I have better ones for this model
        kwargs.setdefault('qk_scale', 768 ** -0.5)
    model = VisionTransformer(patch_size=16, embed_dim=768, depth=8, num_heads=8, mlp_ratio=3., **kwargs)
    model.default_cfg = default_cfgs['vit_small_patch16_224']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter)
    return model


def vit_base_patch16_224(pretrained=False, **kwargs):
    model = VisionTransformer(
        patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_cfgs['vit_base_patch16_224']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter)
    return model


def vit_base_patch16_384(pretrained=False, **kwargs):
    model = VisionTransformer(
        img_size=384, patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_cfgs['vit_base_patch16_384']
    if pretrained:
        load_pretrained(model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3))
    return model


def vit_base_patch32_384(pretrained=False, **kwargs):
    model = VisionTransformer(
        img_size=384, patch_size=32, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_cfgs['vit_base_patch32_384']
    if pretrained:
        load_pretrained(model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3))
    return model


def vit_large_patch16_224(pretrained=False, **kwargs):
    model = VisionTransformer(
        patch_size=16, embed_dim=1024, depth=24, num_heads=16, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_cfgs['vit_large_patch16_224']
    if pretrained:
        load_pretrained(model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3))
    return model


def vit_large_patch16_384(pretrained=False, **kwargs):
    model = VisionTransformer(
        img_size=384, patch_size=16, embed_dim=1024, depth=24, num_heads=16, mlp_ratio=4,  qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_cfgs['vit_large_patch16_384']
    if pretrained:
        load_pretrained(model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3))
    return model


def vit_large_patch32_384(pretrained=False, **kwargs):
    model = VisionTransformer(
        img_size=384, patch_size=32, embed_dim=1024, depth=24, num_heads=16, mlp_ratio=4,  qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_cfgs['vit_large_patch32_384']
    if pretrained:
        load_pretrained(model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3))
    return model


def vit_huge_patch16_224(pretrained=False, **kwargs):
    model = VisionTransformer(patch_size=16, embed_dim=1280, depth=32, num_heads=16, mlp_ratio=4, **kwargs)
    model.default_cfg = default_cfgs['vit_huge_patch16_224']
    return model


def vit_huge_patch32_384(pretrained=False, **kwargs):
    model = VisionTransformer(
        img_size=384, patch_size=32, embed_dim=1280, depth=32, num_heads=16, mlp_ratio=4, **kwargs)
    model.default_cfg = default_cfgs['vit_huge_patch32_384']
    return model


def deit_tiny_patch16_224(pretrained=False, **kwargs):
    model = VisionTransformer(
        patch_size=16, embed_dim=192, depth=12, num_heads=3, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_cfgs['deit_tiny_patch16_224']
    if pretrained:
        checkpoint = torch.hub.load_state_dict_from_url(
            url="https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth",
            map_location="cpu", check_hash=True
        )
        model.load_state_dict(checkpoint["model"])
        print('==>[Loaded PyTorch-pretrained deit checkpoint.]')
    return model


def deit_small_patch16_224(pretrained=False, **kwargs):
    model = VisionTransformer(
        patch_size=16, embed_dim=384, depth=12, num_heads=6, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_cfgs['deit_small_patch16_224']
    if pretrained:
        checkpoint = torch.hub.load_state_dict_from_url(
            url="https://dl.fbaipublicfiles.com/deit/deit_small_patch16_224-cd65a155.pth",
            map_location="cpu", check_hash=True
        )
        model.load_state_dict(checkpoint["model"])
        print('==>[Loaded PyTorch-pretrained deit checkpoint.]')
    return model


def deit_base_patch16_224(pretrained=False, **kwargs):
    model = VisionTransformer(
        patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_cfgs['deit_base_patch16_224']
    if pretrained:
        checkpoint = torch.hub.load_state_dict_from_url(
            url="https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth",
            map_location="cpu", check_hash=True
        )
        model.load_state_dict(checkpoint["model"])
        print('==>[Loaded PyTorch-pretrained deit checkpoint.]')
    return model

def deit_base_patch16_384(pretrained=False, **kwargs):
    model = VisionTransformer(
        img_size=384, patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_cfgs['deit_base_patch16_384']
    if pretrained:
        checkpoint = torch.hub.load_state_dict_from_url(
            url="https://dl.fbaipublicfiles.com/deit/deit_base_patch16_384-8de9b5d1.pth",
            map_location="cpu", check_hash=True
        )
        model.load_state_dict(checkpoint["model"])
        print('==>[Loaded PyTorch-pretrained deit checkpoint.]')
    return model


### CIFAR 10 
def deit_tiny_patch4_32(pretrained=False, **kwargs):
    model = VisionTransformer(
        img_size=32, patch_size=4, embed_dim=192, depth=12, num_heads=3, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model


def deit_small_patch4_32(pretrained=False, **kwargs):
    model = VisionTransformer(
        img_size=32, patch_size=4, embed_dim=384, depth=12, num_heads=6, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model


def deit_base_patch4_32(pretrained=False, **kwargs):
    model = VisionTransformer(
        img_size=32, patch_size=4, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model

## transfer_utils.py

In [5]:
import os 
from robustness import datasets, model_utils

from torchvision import models
from torchvision.datasets import CIFAR100
import torch as ch


pytorch_models = {
    'alexnet': models.alexnet,
    'vgg16': models.vgg16,
    'vgg16_bn': models.vgg16_bn,
    'squeezenet': models.squeezenet1_0,
    'densenet': models.densenet161,
    'shufflenet': models.shufflenet_v2_x1_0,
    'mobilenet': models.mobilenet_v2,
    'resnext50_32x4d': models.resnext50_32x4d,
    'mnasnet': models.mnasnet1_0
}

vitmodeldict = {
    # ImageNet
    'deit_tiny_patch16_224': deit_tiny_patch16_224,
    'deit_small_patch16_224': deit_small_patch16_224,
    'deit_base_patch16_224': deit_base_patch16_224,
    'deit_base_patch16_384': deit_base_patch16_384,
    ##CIFAR10
    'deit_tiny_patch4_32': deit_tiny_patch4_32,
    'deit_small_patch4_32': deit_small_patch4_32,
    'deit_base_patch4_32': deit_base_patch4_32,
}

TRANSFER_DATASETS = ['cifar10', 'cifar100']

def get_dataset_and_loaders(args, shuffle_train=True, shuffle_val=False):
    '''Given arguments, returns a datasets object and the train and validation loaders.
    '''
    ds = datasets.CIFAR(args.data)
    ds.transform_train = TRAIN_TRANSFORMS
    ds.transform_test = TEST_TRANSFORMS
    img_size = 32

    train_loader, val_loader = ds.make_loaders(only_val=args.eval_only, batch_size=args.batch_size, 
                                    workers=args.workers, shuffle_train=shuffle_train, shuffle_val=shuffle_val)
    return ds, train_loader, val_loader


def resume_finetuning_from_checkpoint(args, ds, finetuned_model_path):
    '''Given arguments, dataset object and a finetuned model_path, returns a model
    with loaded weights and returns the checkpoint necessary for resuming training.
    '''
    print('[Resuming finetuning from a checkpoint...]')
    arch, add_custom_forward = get_arch(args)
    if args.dataset in TRANSFER_DATASETS:
        model, _ = model_utils.make_and_restore_model(
            arch=arch, dataset=datasets.ImageNet(''), add_custom_forward=add_custom_forward)
        while hasattr(model, 'model'):
            model = model.model
        model = ft(
            args.arch, model, ds.num_classes, args.additional_hidden)
        model, checkpoint = model_utils.make_and_restore_model(arch=model, dataset=ds, resume_path=finetuned_model_path,
                                                               add_custom_forward=args.additional_hidden > 0 or add_custom_forward)
    else:
        model, checkpoint = model_utils.make_and_restore_model(
            arch=arch, dataset=ds, resume_path=finetuned_model_path,
            add_custom_forward=add_custom_forward)
    return model, checkpoint


def get_arch(args):
    add_custom_forward = True
    if args.arch in pytorch_models.keys():
        arch = pytorch_models[args.arch](args.pytorch_pretrained)
    elif args.arch in vitmodeldict:
        arch = vitmodeldict[args.arch](pretrained=args.pytorch_pretrained,
                                        num_classes=1000,
                                        drop_rate=0.,
                                        drop_path_rate=0.1)
    else:
        arch = args.arch
        add_custom_forward = False
    return arch, add_custom_forward

def get_model(args, ds):
    '''Given arguments and a dataset object, returns an ImageNet model (with appropriate last layer changes to 
    fit the target dataset) and a checkpoint. The checkpoint is set to None if not resuming training.
    '''
  
    finetuned_model_path = os.path.join(
        args.out_dir, args.exp_name, args.resume_ckpt_name)

    if args.resume and os.path.isfile(finetuned_model_path):
        # fix hijacking of normalizer
        patch_state_dict(finetuned_model_path)
        model, checkpoint = resume_finetuning_from_checkpoint(
            args, ds, finetuned_model_path)
    else:
        arch, add_custom_forward = get_arch(args)
        if args.dataset in TRANSFER_DATASETS:
            model, _ = model_utils.make_and_restore_model(
                arch=arch,
                dataset=datasets.ImageNet(''), resume_path=args.model_path, pytorch_pretrained=args.pytorch_pretrained,
                add_custom_forward=add_custom_forward)
            checkpoint = None
        else:
            model, _ = model_utils.make_and_restore_model(arch=arch, dataset=ds,
                                                resume_path=args.model_path, pytorch_pretrained=args.pytorch_pretrained,
                                                add_custom_forward=add_custom_forward)
            checkpoint = None

        if not args.no_replace_last_layer and not args.eval_only and args.dataset in TRANSFER_DATASETS:
            print(f'[Replacing the last layer with {args.additional_hidden} '
                  f'hidden layers and 1 classification layer that fits the {args.dataset} dataset.]')
            while hasattr(model, 'model'):
                model = model.model
            model = ft(
                args.arch, model, ds.num_classes, args.additional_hidden)
            model, checkpoint = model_utils.make_and_restore_model(arch=model, dataset=ds,
                                                                   add_custom_forward=args.additional_hidden > 0 or add_custom_forward)
        else:
            print('[NOT replacing the last layer]')
    return model, checkpoint


def freeze_model(model, freeze_level):
    '''
    Freezes up to args.freeze_level layers of the model (assumes a resnet model)
    '''
    # Freeze layers according to args.freeze-level
    update_params = None
    if freeze_level != -1:
        # assumes a resnet architecture
        assert len([name for name, _ in list(model.named_parameters())
                    if f"layer{freeze_level}" in name]), "unknown freeze level (only {1,2,3,4} for ResNets)"
        update_params = []
        freeze = True
        for name, param in model.named_parameters():
            print(name, param.size())

            if not freeze and f'layer{freeze_level}' not in name:
                print(f"[Appending the params of {name} to the update list]")
                update_params.append(param)
            else:
                param.requires_grad = False

            if freeze and f'layer{freeze_level}' in name:
                # if the freeze level is detected stop freezing onwards
                freeze = False
    return update_params

def patch_state_dict(path): 
    pth = torch.load(path)
    d = pth['model']

    if ("normalizer.1.new_mean" in d or "normalizer.1.new_std" in d
        or "module.normalizer.1.new_mean" in d 
        or "module.normalizer.1.new_std" in d
        or "normalizer.normalizer.new_mean" in d 
        or "normalizer.normalizer.new_std" in d
        or "module.normalizer.normalizer.new_mean" in d 
        or "module.normalizer.normalizer.new_std" in d): 
        print("Patching normalizer module")
        new_d = {}
        for k in d: 
            new_k = k
            if k == "normalizer.1.new_mean": 
                new_k = "normalizer.new_mean"
            if k == "normalizer.1.new_std": 
                new_k = "normalizer.new_std"
            if k == "module.normalizer.1.new_mean": 
                new_k = "module.normalizer.new_mean"
            if k == "module.normalizer.1.new_std": 
                new_k = "module.normalizer.new_std"
            if k == "normalizer.normalizer.new_mean": 
                new_k = "normalizer.new_mean"
            if k == "normalizer.normalizer.new_std": 
                new_k = "normalizer.new_std"
            if k == "module.normalizer.normalizer.new_mean": 
                new_k = "module.normalizer.new_mean"
            if k == "module.normalizer.normalizer.new_std": 
                new_k = "module.normalizer.new_std"
            new_d[new_k] = d[k]
        pth['model'] = new_d
        torch.save(pth,path)
    return

## preprocess.py

In [6]:
import torch.nn as nn
import torch
import torch as ch

class StripeAblator(nn.Module):
    def __init__(self, ablation_size, dim=3):
        super().__init__()
        self.ablation_size = ablation_size
        self.dim = dim
            
    def forward(self, x, pos):
        k = self.ablation_size
        dim = self.dim
        total_pos = x.shape[dim]
        if pos + k > total_pos: 
            idx = [slice(None,None,None) if _ != dim else slice(pos+k-total_pos,pos,None) for _ in range(4)]
            x[idx] = 0
        else: 
            left_idx = [slice(None,None,None) if _ != dim else slice(0, pos, None) for _ in range(4)]
            right_idx = [slice(None,None,None) if _ != dim else slice(pos+k, total_pos, None) for _ in range(4)]
            x[left_idx] = 0
            x[right_idx] = 0
        return x

class BlockAblator(nn.Module):
    def __init__(self, ablation_size):
        super().__init__()
        self.ablation_size = ablation_size
            
    def forward(self, x, pos):
        """
        x: input to be ablated
        pos: tuple (idx_x, idx_y) representing the position of ablation to be applied
        returns: ablated image
        """
        assert len(pos) == 2

        k = self.ablation_size
        total_pos = x.shape[-1]
        pos_x, pos_y = pos
        x_orig = x.clone()
        x[:, :, pos_x:(pos_x + k), pos_y:(pos_y + k)] = 0
        if pos_x + k > total_pos and pos_y + k > total_pos:
            x[:, :, 0:(pos_x + k)%total_pos, 0:(pos_y + k)%total_pos] = 0
            x[:, :, 0:(pos_x + k)%total_pos, pos_y:(pos_y + k)] = 0
            x[:, :, pos_x:(pos_x + k), 0:(pos_y + k)%total_pos] = 0
        elif pos_x + k > total_pos:
            x[:, :, 0:(pos_x + k)%total_pos, pos_y:(pos_y + k)] = 0
        elif pos_y + k > total_pos:
            x[:, :, pos_x:(pos_x + k), 0:(pos_y + k)%total_pos] = 0

        return x_orig - x

class Simple224Upsample(nn.Module):
    # go from 32 to 224
    def __init__(self, arch=''):
        super(Simple224Upsample, self).__init__()
        self.upsample = nn.Upsample(mode='nearest', scale_factor=7)
        self.arch = arch
        
    def forward(self, x):
        return self.upsample(x)

class Upsample384AndPad(nn.Module):
    def __init__(self):
        super(Upsample384AndPad, self).__init__()
        self.upsample = nn.Upsample(mode='nearest', scale_factor=8) # 256
        self.zero_pad = torch.nn.ZeroPad2d((384-256)//2) # 64 on each side
    
    def forward(self, x, ones_mask):
        x = self.upsample(x)
        x = self.zero_pad(x)
        return x
    
cifar_upsamples = {
    'simple224': Simple224Upsample,
    'upsample384': Upsample384AndPad,
    'none': None,
}

class MaskProcessor(nn.Module):
    def __init__(self, patch_size=16):
        super().__init__()
        self.avg_pool = nn.AvgPool2d(patch_size)
    
    def forward(self, ones_mask):
        B = ones_mask.shape[0]
        ones_mask = ones_mask[0].unsqueeze(0) # take the first mask
        ones_mask = self.avg_pool(ones_mask)[0]
        ones_mask = torch.where(ones_mask.view(-1) > 0)[0] + 1
        ones_mask = torch.cat([torch.cuda.IntTensor(1).fill_(0), ones_mask]).unsqueeze(0)
        ones_mask = ones_mask.expand(B, -1)
        return ones_mask
    
class PreProcessor(nn.Module):
    def __init__(self, normalizer, ablation_size, upsample_type='none',
                 return_mask=False, do_ablation=True, ablation_type='col', ablation_target=None):
        '''
        normalizer: the normalizer module
        ablation_size: size of ablation
        upsample_type: type of upsample (none, simple224, upsample384)
        return_mask: if true, keep the mask as a fourth channel
        do_ablation: perform the ablation
        ablation_target: the column to ablate. if None, pick a random column
        '''
        super().__init__()
        print({
            "ablation_size": ablation_size,
            "upsample_type": upsample_type,
            "return_mask": return_mask,
            "do_ablation": do_ablation,
            "ablation_target": ablation_target
        })
        if ablation_type == 'col':
            self.ablator = StripeAblator(ablation_size, dim=3)
        elif ablation_type == 'block':
            self.ablator = BlockAblator(ablation_size)
        else:
            raise Exception('Unkown ablation type')

        if upsample_type == 'none':
            self.upsampler = None
        else:
            self.upsampler = cifar_upsamples[upsample_type]()
        self.return_mask = return_mask
        self.normalizer = normalizer
        self.do_ablation = do_ablation
        self.ablation_target = ablation_target
    
    def forward(self, x):
        B, C, H, W = x.shape
        if C == 3:
            # we don't have a mask yet!!
            ones = torch.ones((B, 1, H, W)).cuda()
            x = torch.cat([x, ones], dim=1)
        else:
            assert not self.do_ablation, "cannot do ablation if already passed in ablation mask"
        if self.do_ablation:
            pos = self.ablation_target
            if pos is None:
                if isinstance(self.ablator, StripeAblator):
                    pos = ch.randint(x.shape[3], (1,))
                elif isinstance(self.ablator, BlockAblator):
                    pos = ch.randint(x.shape[3], (2,))
            x = self.ablator(x=x, pos=pos)
        if self.upsampler is not None:
            x = self.upsampler(x)
        x[:, :3] = self.normalizer(x[:, :3]) # normalize
        if self.return_mask:
            return x # WARNING returning 4 channel output
        else:
            return x[:, :3]

## helpers.py

In [7]:
import logging
import math
import torch.utils.model_zoo as model_zoo


_logger = logging.getLogger(__name__)


def load_pretrained(model, cfg=None, num_classes=1000, in_chans=3, filter_fn=None, strict=True):
    if cfg is None:
        cfg = getattr(model, 'default_cfg')
    if cfg is None or 'url' not in cfg or not cfg['url']:
        _logger.warning("Pretrained model URL is invalid, using random initialization.")
        return

    state_dict = model_zoo.load_url(cfg['url'], progress=False, map_location='cpu')

    if filter_fn is not None:
        state_dict = filter_fn(state_dict)

    if in_chans == 1:
        conv1_name = cfg['first_conv']
        _logger.info('Converting first conv (%s) pretrained weights from 3 to 1 channel' % conv1_name)
        conv1_weight = state_dict[conv1_name + '.weight']
        # Some weights are in torch.half, ensure it's float for sum on CPU
        conv1_type = conv1_weight.dtype
        conv1_weight = conv1_weight.float()
        O, I, J, K = conv1_weight.shape
        if I > 3:
            assert conv1_weight.shape[1] % 3 == 0
            # For models with space2depth stems
            conv1_weight = conv1_weight.reshape(O, I // 3, 3, J, K)
            conv1_weight = conv1_weight.sum(dim=2, keepdim=False)
        else:
            conv1_weight = conv1_weight.sum(dim=1, keepdim=True)
        conv1_weight = conv1_weight.to(conv1_type)
        state_dict[conv1_name + '.weight'] = conv1_weight
    elif in_chans != 3:
        conv1_name = cfg['first_conv']
        conv1_weight = state_dict[conv1_name + '.weight']
        conv1_type = conv1_weight.dtype
        conv1_weight = conv1_weight.float()
        O, I, J, K = conv1_weight.shape
        if I != 3:
            _logger.warning('Deleting first conv (%s) from pretrained weights.' % conv1_name)
            del state_dict[conv1_name + '.weight']
            strict = False
        else:
            # NOTE this strategy should be better than random init, but there could be other combinations of
            # the original RGB input layer weights that'd work better for specific cases.
            _logger.info('Repeating first conv (%s) weights in channel dim.' % conv1_name)
            repeat = int(math.ceil(in_chans / 3))
            conv1_weight = conv1_weight.repeat(1, repeat, 1, 1)[:, :in_chans, :, :]
            conv1_weight *= (3 / float(in_chans))
            conv1_weight = conv1_weight.to(conv1_type)
            state_dict[conv1_name + '.weight'] = conv1_weight

    classifier_name = cfg['classifier']
    if num_classes == 1000 and cfg['num_classes'] == 1001:
        # special case for imagenet trained models with extra background class in pretrained weights
        classifier_weight = state_dict[classifier_name + '.weight']
        state_dict[classifier_name + '.weight'] = classifier_weight[1:]
        classifier_bias = state_dict[classifier_name + '.bias']
        state_dict[classifier_name + '.bias'] = classifier_bias[1:]
    elif num_classes != cfg['num_classes']:
        # completely discard fully connected for all other differences between pretrained and created model
        del state_dict[classifier_name + '.weight']
        del state_dict[classifier_name + '.bias']
        strict = False

    model.load_state_dict(state_dict, strict=strict)

## layers/helpers.py 

In [8]:
""" Layer/Module Helpers
Hacked together by / Copyright 2020 Ross Wightman
"""
from itertools import repeat

import collections.abc as container_abcs
# from torch._six import container_abcs


# From PyTorch internals
def _ntuple(n):
    def parse(x):
        if isinstance(x, container_abcs.Iterable):
            return x
        return tuple(repeat(x, n))
    return parse


to_1tuple = _ntuple(1)
to_2tuple = _ntuple(2)
to_3tuple = _ntuple(3)
to_4tuple = _ntuple(4)
to_ntuple = _ntuple

## layers/drop.py

In [9]:
""" DropBlock, DropPath
PyTorch implementations of DropBlock and DropPath (Stochastic Depth) regularization layers.
Papers:
DropBlock: A regularization method for convolutional networks (https://arxiv.org/abs/1810.12890)
Deep Networks with Stochastic Depth (https://arxiv.org/abs/1603.09382)
Code:
DropBlock impl inspired by two Tensorflow impl that I liked:
 - https://github.com/tensorflow/tpu/blob/master/models/official/resnet/resnet_model.py#L74
 - https://github.com/clovaai/assembled-cnn/blob/master/nets/blocks.py
Hacked together by / Copyright 2020 Ross Wightman
"""
import torch
import torch.nn as nn
import torch.nn.functional as F


def drop_block_2d(
        x, drop_prob: float = 0.1, block_size: int = 7,  gamma_scale: float = 1.0,
        with_noise: bool = False, inplace: bool = False, batchwise: bool = False):
    """ DropBlock. See https://arxiv.org/pdf/1810.12890.pdf
    DropBlock with an experimental gaussian noise option. This layer has been tested on a few training
    runs with success, but needs further validation and possibly optimization for lower runtime impact.
    """
    B, C, H, W = x.shape
    total_size = W * H
    clipped_block_size = min(block_size, min(W, H))
    # seed_drop_rate, the gamma parameter
    gamma = gamma_scale * drop_prob * total_size / clipped_block_size ** 2 / (
        (W - block_size + 1) * (H - block_size + 1))

    # Forces the block to be inside the feature map.
    w_i, h_i = torch.meshgrid(torch.arange(W).to(x.device), torch.arange(H).to(x.device))
    valid_block = ((w_i >= clipped_block_size // 2) & (w_i < W - (clipped_block_size - 1) // 2)) & \
                  ((h_i >= clipped_block_size // 2) & (h_i < H - (clipped_block_size - 1) // 2))
    valid_block = torch.reshape(valid_block, (1, 1, H, W)).to(dtype=x.dtype)

    if batchwise:
        # one mask for whole batch, quite a bit faster
        uniform_noise = torch.rand((1, C, H, W), dtype=x.dtype, device=x.device)
    else:
        uniform_noise = torch.rand_like(x)
    block_mask = ((2 - gamma - valid_block + uniform_noise) >= 1).to(dtype=x.dtype)
    block_mask = -F.max_pool2d(
        -block_mask,
        kernel_size=clipped_block_size,  # block_size,
        stride=1,
        padding=clipped_block_size // 2)

    if with_noise:
        normal_noise = torch.randn((1, C, H, W), dtype=x.dtype, device=x.device) if batchwise else torch.randn_like(x)
        if inplace:
            x.mul_(block_mask).add_(normal_noise * (1 - block_mask))
        else:
            x = x * block_mask + normal_noise * (1 - block_mask)
    else:
        normalize_scale = (block_mask.numel() / block_mask.to(dtype=torch.float32).sum().add(1e-7)).to(x.dtype)
        if inplace:
            x.mul_(block_mask * normalize_scale)
        else:
            x = x * block_mask * normalize_scale
    return x


def drop_block_fast_2d(
        x: torch.Tensor, drop_prob: float = 0.1, block_size: int = 7,
        gamma_scale: float = 1.0, with_noise: bool = False, inplace: bool = False, batchwise: bool = False):
    """ DropBlock. See https://arxiv.org/pdf/1810.12890.pdf
    DropBlock with an experimental gaussian noise option. Simplied from above without concern for valid
    block mask at edges.
    """
    B, C, H, W = x.shape
    total_size = W * H
    clipped_block_size = min(block_size, min(W, H))
    gamma = gamma_scale * drop_prob * total_size / clipped_block_size ** 2 / (
            (W - block_size + 1) * (H - block_size + 1))

    if batchwise:
        # one mask for whole batch, quite a bit faster
        block_mask = torch.rand((1, C, H, W), dtype=x.dtype, device=x.device) < gamma
    else:
        # mask per batch element
        block_mask = torch.rand_like(x) < gamma
    block_mask = F.max_pool2d(
        block_mask.to(x.dtype), kernel_size=clipped_block_size, stride=1, padding=clipped_block_size // 2)

    if with_noise:
        normal_noise = torch.randn((1, C, H, W), dtype=x.dtype, device=x.device) if batchwise else torch.randn_like(x)
        if inplace:
            x.mul_(1. - block_mask).add_(normal_noise * block_mask)
        else:
            x = x * (1. - block_mask) + normal_noise * block_mask
    else:
        block_mask = 1 - block_mask
        normalize_scale = (block_mask.numel() / block_mask.to(dtype=torch.float32).sum().add(1e-7)).to(dtype=x.dtype)
        if inplace:
            x.mul_(block_mask * normalize_scale)
        else:
            x = x * block_mask * normalize_scale
    return x


class DropBlock2d(nn.Module):
    """ DropBlock. See https://arxiv.org/pdf/1810.12890.pdf
    """
    def __init__(self,
                 drop_prob=0.1,
                 block_size=7,
                 gamma_scale=1.0,
                 with_noise=False,
                 inplace=False,
                 batchwise=False,
                 fast=True):
        super(DropBlock2d, self).__init__()
        self.drop_prob = drop_prob
        self.gamma_scale = gamma_scale
        self.block_size = block_size
        self.with_noise = with_noise
        self.inplace = inplace
        self.batchwise = batchwise
        self.fast = fast  # FIXME finish comparisons of fast vs not

    def forward(self, x):
        if not self.training or not self.drop_prob:
            return x
        if self.fast:
            return drop_block_fast_2d(
                x, self.drop_prob, self.block_size, self.gamma_scale, self.with_noise, self.inplace, self.batchwise)
        else:
            return drop_block_2d(
                x, self.drop_prob, self.block_size, self.gamma_scale, self.with_noise, self.inplace, self.batchwise)


def drop_path(x, drop_prob: float = 0., training: bool = False):
    """Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)

## layers/weight_init.py

In [10]:
import torch
import math
import warnings


def _no_grad_trunc_normal_(tensor, mean, std, a, b):
    # Cut & paste from PyTorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # type: (Tensor, float, float, float, float) -> Tensor
    r"""Fills the input Tensor with values drawn from a truncated
    normal distribution. The values are effectively drawn from the
    normal distribution :math:`\mathcal{N}(\text{mean}, \text{std}^2)`
    with values outside :math:`[a, b]` redrawn until they are within
    the bounds. The method used for generating the random values works
    best when :math:`a \leq \text{mean} \leq b`.
    Args:
        tensor: an n-dimensional `torch.Tensor`
        mean: the mean of the normal distribution
        std: the standard deviation of the normal distribution
        a: the minimum cutoff value
        b: the maximum cutoff value
    Examples:
        >>> w = torch.empty(3, 5)
        >>> nn.init.trunc_normal_(w)
    """
    return _no_grad_trunc_normal_(tensor, mean, std, a, b)

## GPU info

In [11]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e507df0a-da7a-6312-edb8-c871350fa53d)


## main.py

In [13]:
import argparse
import os
import sys
from datetime import datetime

import cox.store
import numpy as np
import torch as ch
from robustness import datasets, defaults, train

if int(os.environ.get("NOTEBOOK_MODE", 0)) == 1:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm as tqdm

parser = argparse.ArgumentParser(description='Transfer learning via pretrained Imagenet models',
                                 conflict_handler='resolve')
parser = defaults.add_args_to_parser(defaults.CONFIG_ARGS, parser)
parser = defaults.add_args_to_parser(defaults.MODEL_LOADER_ARGS, parser)
parser = defaults.add_args_to_parser(defaults.TRAINING_ARGS, parser)
parser = defaults.add_args_to_parser(defaults.PGD_ARGS, parser)

# Custom arguments
parser.add_argument('--dataset', type=str, default='cifar10',
                    help='Downstream task dataset (Overrides the one in robustness.defaults)')
parser.add_argument('--model-path', type=str, help='Path to model trained with robustness lib')
parser.add_argument('--resume', action='store_true',
                    help='Whether to resume or not (Overrides the one in robustness.defaults)')
parser.add_argument('--resume-ckpt-name', type=str, default='checkpoint.pt.latest', 
                    help='Name of the checkpoint to resume from')
parser.add_argument('--pytorch-pretrained', default=True, action='store_true',
                    help='If True, loads a Pytorch pretrained model.')
parser.add_argument('--subset', type=int, default=None,
                    help='number of training data to use from the dataset')
parser.add_argument('--no-tqdm', type=int, default=0,
                    choices=[0, 1], help='Do not use tqdm.')
parser.add_argument('--no-replace-last-layer', action='store_true',
                    help='Whether to avoid replacing the last layer')
parser.add_argument('--freeze-level', type=int, default=-1,
                    help='Up to what layer to freeze in the pretrained model (assumes a resnet architectures)')
parser.add_argument('--additional-hidden', type=int, default=0,
                    help='How many hidden layers to add on top of pretrained network + classification layer')
parser.add_argument('--update-BN-stats', action='store_true')
parser.add_argument('--cifar-preprocess-type', type=str, default='simple224', 
                    help='What cifar preprocess type to use (only use with CIFAR-10)',
                    choices=['simple224', 'upsample384', 'none'])
parser.add_argument('--drop-tokens', default=True, action='store_true', help='drop masked out tokens (for ViTs only)')
parser.add_argument('--ablation-target', type=int, default=None, help='choose specific column to keep')


## Input Ablation
parser.add_argument('--ablate-input', '--ablate_input', default=True, action='store_true')
parser.add_argument('--ablation-type', '--ablation_type', type=str, default='col',
                    help='Type of ablations', choices=['col', 'block'])
parser.add_argument('--ablation-size', '--ablation_size', type=int, default=4,
                    help='Width of the remaining column if --ablation-type is "col".' 
                    'Side length of the remaining block if --ablation-type is "block"')

# certification arguments
parser.add_argument('--skip-store', default=True, action='store_true')
parser.add_argument('--certify', action='store_true')
parser.add_argument('--certify-out-dir', default='OUTDIR_CERT')
parser.add_argument('--certify-mode', default='both', choices=['both', 'row', 'col', 'block'])
parser.add_argument('--certify-ablation-size', type=int, default=4)
parser.add_argument('--certify-patch-size', type=int, default=5)
parser.add_argument('--certify-stride', type=int, default=1)
parser.add_argument('--batch-id', type=int, default=None)

# Colab Adjustments
parser.add_argument('--out-dir', '--out_dir', default='/content/drive/MyDrive/Seminar1')
parser.add_argument('--adv-train', '--adv_train', default=0)
parser.add_argument('--arch', default='deit_tiny_patch16_224')
parser.add_argument('--exp-name', '--exp_name', default='model')
parser.add_argument('-f', '--fff', help='a dummy argument to fool ipython', default='1')

def main(args, store):
    '''Given arguments and a cox store, trains as a model. Check out the 
    argparse object in this file for argument options.
    '''
    ds, train_loader, validation_loader = get_dataset_and_loaders(args)
    print(args)
    model, checkpoint = get_model(args, ds)

    def get_n_params(model):
        total = 0
        for p in list(model.parameters()):
            total += np.prod(p.size())
        return total
    print(f'==> [Number of parameters of the model is {get_n_params(model)}]')
    
    model.normalizer = PreProcessor(
        normalizer=model.normalizer,
        ablation_size=args.ablation_size, 
        upsample_type=args.cifar_preprocess_type,
        return_mask=args.drop_tokens,
        do_ablation=args.ablate_input,
        ablation_type=args.ablation_type,
        ablation_target=args.ablation_target,
    )
    if 'deit' not in args.arch:
        assert args.drop_tokens == False
    
    if args.update_BN_stats:
        print(f'==>[Started updating the BN stats relevant to {args.dataset}]')
        assert not hasattr(model, "module"), "model is already in DataParallel."
        model = model.cuda()
        model.train()
        attack_kwargs = {
            'constraint': args.constraint,
            'eps': args.eps,
            'step_size': args.attack_lr,
            'iterations': args.attack_steps,
        }
        with ch.no_grad():
            niters = 0
            while niters < 200:
                iterator = tqdm(enumerate(train_loader), total=len(train_loader))
                for _, (inp, _) in iterator:
                    model(inp.cuda(), **attack_kwargs)
                    niters += 1
        print('==>[Updated the BN stats]')

    if args.eval_only:
        if args.certify: 
            return certify(args, model, validation_loader, store=store)
        else: 
            return train.eval_model(args, model, validation_loader, store=store)

    update_params = freeze_model(model, freeze_level=args.freeze_level)
    print(f"Dataset: {args.dataset} | Model: {args.arch}")

    train.train_model(args, model, (train_loader, validation_loader), store=store,
                        checkpoint=checkpoint, update_params=update_params)


def args_preprocess(args):
    '''
    Fill the args object with reasonable defaults, and also perform a sanity check to make sure no
    args are missing.
    '''
    if args.adv_train and eval(args.eps) == 0:
        print('[Switching to standard training since eps = 0]')
        args.adv_train = 0

    if args.pytorch_pretrained:
        assert not args.model_path, 'You can either specify pytorch_pretrained or model_path, not together.'


    ALL_DS = TRANSFER_DATASETS + ['imagenet', 'stylized_imagenet']


    #Prepare training args
    # Preprocess args
    default_ds = args.dataset if args.dataset in datasets.DATASETS else "cifar"
    args = defaults.check_and_fill_args(args, defaults.CONFIG_ARGS, datasets.CIFAR)
    if not args.eval_only:
        args = defaults.check_and_fill_args(args, defaults.TRAINING_ARGS, datasets.CIFAR)
    if args.adv_train or args.adv_eval:
        args = defaults.check_and_fill_args(args, defaults.PGD_ARGS, datasets.CIFAR)
    args = defaults.check_and_fill_args(args, defaults.MODEL_LOADER_ARGS, datasets.CIFAR)

    return args


args = parser.parse_args()
args = args_preprocess(args)
if args.skip_store: 
    store = None

main(args, store)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 30 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Namespace(ablate_input=True, ablation_size=4, ablation_target=None, ablation_type='col', additional_hidden=0, adv_eval=None, adv_train=0, arch='deit_tiny_patch16_224', attack_lr=None, attack_steps=None, batch_id=None, batch_size=128, certify=False, certify_ablation_size=4, certify_mode='both', certify_out_dir='OUTDIR_CERT', certify_patch_size=5, certify_stride=1, cifar_preprocess_type='simple224', config_path=None, constraint=None, custom_eps_multiplier=None, custom_lr_multiplier=None, data='/tmp/', data_aug=1, dataset='cifar10', drop_tokens=True, epochs=30, eps=None, eval_only=0, exp_name='model', fff='/root/.local/share/jupyter/runtime/kernel-0626abb0-4c9f-4738-8741-63f5a26a5089.json', freeze_level=-1, log_iters=5, lr=0.01, lr_interpolation='step', mixed_precision=0, model_path=None, momentum=0.9, no_replace_last_layer=False, no_tqdm=0, out_dir='/content/drive/MyDrive/Seminar1', pytorch_pretrained=True, random_restarts=None, random_start=None, resume=False, resume_ckpt_name='checkpoi

  0%|          | 0/391 [00:00<?, ?it/s]

USING MISSINGNESS


Train Epoch:0 | Loss 1.7092 | NatPrec1 37.628 | NatPrec5 85.152 | Reg term: 0.0 ||: 100%|██████████| 391/391 [00:41<00:00,  9.40it/s]
Val Epoch:0 | Loss 1.4399 | NatPrec1 48.330 | NatPrec5 92.080 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:04<00:00, 16.03it/s]
Train Epoch:1 | Loss 1.3445 | NatPrec1 52.074 | NatPrec5 92.930 | Reg term: 0.0 ||: 100%|██████████| 391/391 [00:40<00:00,  9.75it/s]
Train Epoch:2 | Loss 1.2505 | NatPrec1 55.496 | NatPrec5 94.094 | Reg term: 0.0 ||: 100%|██████████| 391/391 [00:41<00:00,  9.46it/s]
Train Epoch:3 | Loss 1.1865 | NatPrec1 57.616 | NatPrec5 94.872 | Reg term: 0.0 ||: 100%|██████████| 391/391 [00:42<00:00,  9.23it/s]
Train Epoch:4 | Loss 1.1314 | NatPrec1 59.862 | NatPrec5 95.390 | Reg term: 0.0 ||: 100%|██████████| 391/391 [00:42<00:00,  9.28it/s]
Train Epoch:5 | Loss 1.1056 | NatPrec1 60.870 | NatPrec5 95.428 | Reg term: 0.0 ||: 100%|██████████| 391/391 [00:42<00:00,  9.31it/s]
Val Epoch:5 | Loss 1.0886 | NatPrec1 61.480 | NatPrec5 95.780 | Re